# Import des modules nécessaires

In [2]:
    # Pour manipuler efficacement des tables de données dans Python
    import pandas as pd

    # Pour faire des requêtes GET et POST
    import requests

In [3]:
print(pd.__version__)

0.23.0


In [4]:
print(requests.__version__)

2.18.4


# Requêtage d'une API REST

Pour lancer une requête GET, c'est ultra-simple : on utilise `requests.get(url)`.

Si vous voulez glisser des paramètres avec la requête, utilisez l'argument optionnel `params`, qui accepte un simple dictionnaire.

In [5]:
r = requests.get("https://data.angers.fr/api/records/1.0/search/",
                 params={
                     'dataset': 'horaires-theoriques-et-arrets-du-reseau-irigo-gtfs',
                     'rows': 10
                 })

In [6]:
g = requests.get("https://data.angers.fr/api/records/1.0/search/",
                 params={
                     'dataset': 'bus-tram-position-tr',
                     'rows': 200
                 })

Regardons la nature de ce qui nous est renvoyé.

In [7]:
type(g)

requests.models.Response

C'est un objet de la classe `Response`. Si vous voulez connaitre toutes les méthodes attachées, faites `help(r)`

In [8]:
help(g)

Help on Response in module requests.models object:

class Response(builtins.object)
 |  The :class:`Response <Response>` object, which contains a
 |  server's response to an HTTP request.
 |  
 |  Methods defined here:
 |  
 |  __bool__(self)
 |      Returns True if :attr:`status_code` is less than 400.
 |      
 |      This attribute checks if the status code of the response is between
 |      400 and 600 to see if there was a client error or a server error. If
 |      the status code, is between 200 and 400, this will return True. This
 |      is **not** a check to see if the response code is ``200 OK``.
 |  
 |  __enter__(self)
 |  
 |  __exit__(self, *args)
 |  
 |  __getstate__(self)
 |  
 |  __init__(self)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __iter__(self)
 |      Allows you to use a response as an iterator.
 |  
 |  __nonzero__(self)
 |      Returns True if :attr:`status_code` is less than 400.
 |      
 |      This attribute checks if

Pour tester que tout a bien fonctionnné, on va vérifié le statut.

In [9]:
g.raise_for_status()

Si ça dit rien c'est que c'est bon :) Dans le doute, regardons le statut...

In [10]:
g.status_code

200

Statut 200 = succès !!

Pour récupérer le contenu de la réponse, vous pouvez utiliser `r.text`.

Il y a aussi un décodeur JSON, si on sait que la réponse est dans ce format. Du coup, il parse le JSON et rend un dictionnaire !

In [11]:
g.json()

{'nhits': 134,
 'parameters': {'dataset': ['bus-tram-position-tr'],
  'timezone': 'UTC',
  'rows': 200,
  'format': 'json'},
 'records': [{'datasetid': 'bus-tram-position-tr',
   'recordid': '7d38ad93bae9122ed8d759dbf052ddecb200ded5',
   'fields': {'etat': 'LIGN',
    'type': 'GX 427 Hyb',
    'ecart': 338,
    'novh': '201',
    'dest': 'BEAUCOUZE  HAUTE ROCHE',
    'idparcours': 268741632,
    'cap': 6,
    'nomarret': 'GAUBERT',
    'idvh': 268435657,
    'iddesserte': 268741673,
    'mnemoligne': '04',
    'y': 2278950,
    'nomligne': 'BEAUCOUZE <> ST BARTHELEMY',
    'mnemoarret': 'GAUBER-E',
    'idarret': 669723,
    'harret': '2018-06-18T13:59:43+00:00',
    'x': 378709,
    'sv': '0407',
    'coordonnees': [47.472957, -0.59988059],
    'idligne': 268435460,
    'numarret': 12547},
   'geometry': {'type': 'Point', 'coordinates': [-0.59988059, 47.472957]},
   'record_timestamp': '2018-06-18T13:58:35+00:00'},
  {'datasetid': 'bus-tram-position-tr',
   'recordid': '4e6a006e5d11d4

# Remplissage de DataFrame

On peut construire une `DataFrame` directement à partir d'un "dictionnaires de colonnes".

In [12]:
df = pd.DataFrame({
    'a': [1, 2, 3],
    'b': [3, 4, 1]
})

df

,a,b
0,1,3
1,2,4
2,3,1


Par exemple, listons les arrêts, leur id, leur nom et leurs coordonnées GPS.

In [13]:
dd = g.json()['records']

In [14]:
dd[0]

{'datasetid': 'bus-tram-position-tr',
 'recordid': '7d38ad93bae9122ed8d759dbf052ddecb200ded5',
 'fields': {'etat': 'LIGN',
  'type': 'GX 427 Hyb',
  'ecart': 338,
  'novh': '201',
  'dest': 'BEAUCOUZE  HAUTE ROCHE',
  'idparcours': 268741632,
  'cap': 6,
  'nomarret': 'GAUBERT',
  'idvh': 268435657,
  'iddesserte': 268741673,
  'mnemoligne': '04',
  'y': 2278950,
  'nomligne': 'BEAUCOUZE <> ST BARTHELEMY',
  'mnemoarret': 'GAUBER-E',
  'idarret': 669723,
  'harret': '2018-06-18T13:59:43+00:00',
  'x': 378709,
  'sv': '0407',
  'coordonnees': [47.472957, -0.59988059],
  'idligne': 268435460,
  'numarret': 12547},
 'geometry': {'type': 'Point', 'coordinates': [-0.59988059, 47.472957]},
 'record_timestamp': '2018-06-18T13:58:35+00:00'}

In [15]:
#Arret
id_arret = [elem['fields']['idarret'] for elem in dd]
nom_arret = [elem['fields']['nomarret'] for elem in dd]
mne_arret = [elem['fields']['mnemoarret'] for elem in dd]

arret = pd.DataFrame({
    'id_arret': id_arret,
    'nom_arret': nom_arret,
    'mne_arret': mne_arret
})

arret.info()

arret.sample(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134 entries, 0 to 133
Data columns (total 3 columns):
id_arret     134 non-null int64
nom_arret    134 non-null object
mne_arret    134 non-null object
dtypes: int64(1), object(2)
memory usage: 3.2+ KB


,id_arret,nom_arret,mne_arret
91,669474,CAMELIA,CAMELI-E
102,669626,DIONNEAU,DIONNE-E
26,669397,ST AUBIN LA SALLE,BARONNER
101,670094,PLACE DE L'EUROPE,PLDEUR-E
61,669458,BOURSE,BOURSE-E
83,670819,AQUA VITA,AQUA-D5A
56,669567,CLINIQUE DE L'ANJOU,CLINANJO
118,670214,SORTIE TRAM CTT,SDPTRAM
130,669535,CHATEAU D'ANGERS,CHDARR-E
115,669953,MAIRIE PONTS DE CE,MAPTSCE


In [16]:
#Bus
id_vh = [elem['fields']['idvh'] for elem in dd]
type_vh = [elem['fields']['type'] for elem in dd]
etat_vh = [elem['fields']['etat'] for elem in dd]

bus = pd.DataFrame({
    'id_vh': id_vh,
    'type_vh': type_vh,
    'etat_vh': etat_vh
})

bus.info()

bus.sample(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134 entries, 0 to 133
Data columns (total 5 columns):
id_vh        134 non-null int64
type_vh      134 non-null object
etat_vh      134 non-null object
latitude     134 non-null float64
longitude    134 non-null float64
dtypes: float64(2), int64(1), object(2)
memory usage: 5.3+ KB


,id_vh,type_vh,etat_vh,latitude,longitude
26,268435918,OMNICITY,TDEP,47.489367,-0.503753
62,268435991,GX 327,LIGN,47.487193,-0.552383
117,268436134,OmniArtE6,LIGN,47.445524,-0.465380
95,268436111,OMNIART,LIGN,47.400466,-0.543098
14,268436107,OMNIART,LIGN,47.492487,-0.502432
13,268436079,MAN City,LIGN,47.489261,-0.584087
85,268435887,OMNICITY,LIGN,47.470001,-0.618409
81,268436124,OMNIART,TDEP,47.472720,-0.485927
48,268436473,CITADIS,LIGN,47.493548,-0.576115
110,268436459,CITADIS,LIGN,47.508634,-0.591461


In [19]:
#Ligne
id_ligne = [elem['fields']['idligne'] for elem in dd]
nom_ligne = [elem['fields']['nomligne'] for elem in dd]
mne_ligne = [elem['fields']['mnemoligne'] for elem in dd]

ligne = pd.DataFrame({
    'id_ligne': id_arret,
    'nom_ligne': nom_ligne,
    'mne_ligne': mne_ligne
})

ligne.info()

ligne

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134 entries, 0 to 133
Data columns (total 3 columns):
id_ligne     134 non-null int64
nom_ligne    134 non-null object
mne_ligne    134 non-null object
dtypes: int64(1), object(2)
memory usage: 3.2+ KB


,id_ligne,nom_ligne,mne_ligne
0,669723,BEAUCOUZE <> ST BARTHELEMY,04
1,669609,ST LEZIN SORGES <> SCHWEITZER,10
2,669481,HOPITAL <> MONTREUIL JUIGNE,07
3,669332,HOPITAL <> MONTREUIL JUIGNE,07
4,669615,ST LEZIN SORGES <> SCHWEITZER,10
5,669917,M-MARCILLE <> ST AUBIN LA SALLE,12
6,669475,CIRCULAIRE VERNEAU GARE EUROPE,05
7,670213,BEAUCOUZE <> ST BARTHELEMY,04
8,669447,BELLE BEILLE <> MONPLAISIR,01
9,670290,ST SYLVAIN BANCHAIS <>TRELAZE,02


In [25]:
#Trajet
id_bus = [elem['fields']['idvh'] for elem in dd]
id_ligne = [elem['fields']['idligne'] for elem in dd]
latitude = [elem['fields']['coordonnees'][0] for elem in dd]
longitude = [elem['fields']['coordonnees'][1] for elem in dd]

trajet = pd.DataFrame({
    'id_bus': id_bus,
    'id_ligne': id_ligne,
    'latitude': latitude,
    'longitude': longitude
})

trajet.info()

trajet

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134 entries, 0 to 133
Data columns (total 4 columns):
id_bus       134 non-null int64
id_ligne     134 non-null int64
latitude     134 non-null float64
longitude    134 non-null float64
dtypes: float64(2), int64(2)
memory usage: 4.3 KB


,id_bus,id_ligne,latitude,longitude
0,268435657,268435460,47.472957,-0.599881
1,268435881,268435466,47.452383,-0.542029
2,268435884,268435463,47.538794,-0.616369
3,268435899,268435463,47.511879,-0.596595
4,268435905,268435466,47.475039,-0.493046
5,268435987,268435468,47.470265,-0.547217
6,268435994,268435461,47.470320,-0.519211
7,268436057,268435460,47.496393,-0.568169
8,268436059,268435457,47.478236,-0.584757
9,268436064,268435458,47.450235,-0.517192


In [29]:
#Etape
id_arret = [elem['fields']['idarret'] for elem in dd]
harret = [elem['fields']['harret'] for elem in dd]
record_timestamp = [elem['record_timestamp'] for elem in dd]
ecart = [elem['fields']['ecart'] for elem in dd]

def transfo(row):
    return record_timestamp + dt.timedelta(seconds=ecart)

heure_estime = df.apply(transfo, axis='columns')

etape = pd.DataFrame({
    'id_arret': id_arret,
    'heure_theorique': harret,
    'heure_estime': heure_estime,
    'ecart': ecart
})

etape.info()

etape

NameError: ("name 'dt' is not defined", 'occurred at index 0')